In [1]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import json

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
proj_dir = Path("../../../..")

data_dir = proj_dir / "Data/InSituTemperature"
data_dir.mkdir(exist_ok=True)

# load metadata
metadata = json.load(Path(data_dir, "processed", "metadata.json").open("r"))
usbr_stations_metadata = json.load(Path('stations.json').open("r"))

pcode_keys = usbr_stations_metadata["pcode_keys"]

In [3]:
# convert from deg min sec to decimal degrees
def dms2dd(degrees, minutes=0, seconds=0, direction=None):
    dd = float(degrees) + float(minutes) / 60 + float(seconds) / (60 * 60)
    if direction == "S" or direction == "W":
        dd *= -1
    return dd

In [4]:
"-2-4-3".strip("-")

'2-4-3'

In [5]:
# load the list of file in the processed directory
files = [f for f in Path(data_dir, "processed").glob("*.csv")]

# loop through the files and update the metadata
for f in files:
    station = f.stem
    station_id = station.split("_")[1]

    if station_id in usbr_stations_metadata.keys():
        if station not in metadata["stations"].keys():
            metadata["stations"][station] = {}

        metadata["stations"][station]["source"] = "USBR"
        metadata["stations"][station]["id"] = station_id
        metadata["stations"][station]["description"] = usbr_stations_metadata[
            station_id
        ]["description"]
        # print(usbr_stations_metadata[station_id]["latitude"].split("-"))
        metadata["stations"][station]["latitude"] = dms2dd(
            *usbr_stations_metadata[station_id]["latitude"].strip("-").split("-"), direction="N"
        )
        metadata["stations"][station]["longitude"] = dms2dd(
            direction="W",*usbr_stations_metadata[station_id]["longitude"].strip("-").split("-"), 
        )
        metadata["stations"][station]["elevation"] = usbr_stations_metadata[station_id][
            "elevation"
        ]
        # update the parameters instead of overwriting
        if "parameters" not in metadata["stations"][station].keys():
            metadata["stations"][station]["parameters"] = {}
        for p in usbr_stations_metadata[station_id]["pcodes"]:
            if p in pcode_keys.keys():
                metadata["stations"][station]["parameters"][pcode_keys[p]["column_name"]] =pcode_keys[p]["description"]
        
        # TODO: update the parameters instead of overwriting them
        if "parameters" not in metadata["stations"][station].keys():
            metadata["stations"][station]["parameters"] = {}
        for p in usbr_stations_metadata[station_id]["pcodes"]:
            if p in pcode_keys.keys():
                metadata["stations"][station]["parameters"][pcode_keys[p]["column_name"]] =pcode_keys[p]["description"]

        # metadata["stations"][station]["parameters"] = {
        #     pcode_keys[p]["column_name"]: pcode_keys[p]["description"]
        #     for p in usbr_stations_metadata[station_id]["pcodes"]
        #     if p in pcode_keys.keys()
        # }
        # add geometry
        metadata["stations"][station]["geometry"] = {
            "type": "Point",
            "coordinates": [
                metadata["stations"][station]["longitude"],
                metadata["stations"][station]["latitude"],
            ],
        }

# add last updated date and last updated by
metadata["last_updated"] = pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")
metadata["last_updated_by"] = "George Darkwah"
metadata["last_updated_by_email"] = "gdarkwah@uw.edu"

# save the metadata
json.dump(metadata, Path(data_dir, "processed", "metadata.json").open("w"), indent=4)

In [6]:
metadata["last_updated"] = pd.Timestamp.now().strftime("%Y-%m-%d")
metadata["last_updated_by"] = "George Darkwah"